# Classification regression

In this example, I will be folloing the example at page-158 of chapter-5 on the 'German credit rating dataset'. 

**About the datasets:** German credit rating dataset is available at the University of California
Irvine (UCI) machine learning laboratory to predict whether a credit is a good or bad credit. Credit
classification is one of the popular classification problems encountered by financial institutions. For
example, when a customer applies for a loan, financial institutions such as banks would like to predict
the probability of default. In this jupyter notebook, we will be using the German credit rating dataset provided by Prof. Hofmann (available at UCI machine learning data repository). 

The data contains several attributes of the persons who availed the credit.

https://archive.ics.uci.edu/dataset/144/statlog+german+credit+data

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [2]:
credit_df = pd.read_csv('D:\Arun 2022\Github\Machine-Learning\ml-data\German Credit Data.csv')

In [3]:
credit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   checkin_acc        1000 non-null   object
 1   duration           1000 non-null   int64 
 2   credit_history     1000 non-null   object
 3   amount             1000 non-null   int64 
 4   savings_acc        1000 non-null   object
 5   present_emp_since  1000 non-null   object
 6   inst_rate          1000 non-null   int64 
 7   personal_status    1000 non-null   object
 8   residing_since     1000 non-null   int64 
 9   age                1000 non-null   int64 
 10  inst_plans         1000 non-null   object
 11  num_credits        1000 non-null   int64 
 12  job                1000 non-null   object
 13  status             1000 non-null   int64 
dtypes: int64(7), object(7)
memory usage: 109.5+ KB


The dataset contains 1000 records and 14 columns. 

In [5]:
credit_df.iloc[0:5, 1:7]

,duration,credit_history,amount,savings_acc,present_emp_since,inst_rate
0,6,A34,1169,A65,A75,4
1,48,A32,5951,A61,A73,2
2,12,A34,2096,A61,A74,2
3,42,A32,7882,A61,A74,2
4,24,A33,4870,A61,A73,3


In [6]:
credit_df.head()

,checkin_acc,duration,credit_history,amount,savings_acc,present_emp_since,inst_rate,personal_status,residing_since,age,inst_plans,num_credits,job,status
0,A11,6,A34,1169,A65,A75,4,A93,4,67,A143,2,A173,0
1,A12,48,A32,5951,A61,A73,2,A92,2,22,A143,1,A173,1
2,A14,12,A34,2096,A61,A74,2,A93,3,49,A143,1,A172,0
3,A11,42,A32,7882,A61,A74,2,A93,4,45,A143,1,A173,0
4,A11,24,A33,4870,A61,A73,3,A93,4,53,A143,2,A173,1


In [11]:
credit_df['checkin_acc'].unique()

array(['A11', 'A12', 'A14', 'A13'], dtype=object)

There are few columns which are categorical and have been inferred as objects. For example, checkin_acc
has following categories:


| Credit Rating	| Description | Money |
|---------------|-------------|-------|
| A11	| Very poor creditworthiness. The debtor has no or negative assets. | No checking account |
| A12	| Poor creditworthiness. The debtor has some assets, but these are not sufficient to cover their liabilities. | Checking account with < 200 DM |
| A13	| Good creditworthiness. The debtor has sufficient assets to cover their liabilities. They have a regular income from salary assignments and have been receiving these payments for at least one year. | Checking account with >= 200 DM and at least one year of salary assignments |
| A14	| No checking account. The debtor does not have a checking account. This can make it difficult for them to manage their finances and may indicate that they are financially unstable. | Checking account with >= 200 DM and no salary assignments |

(above 'DM' represents `Deutsche Mark`)

In [10]:
credit_df['job'].unique()

array(['A173', 'A172', 'A174', 'A171'], dtype=object)

- A173: Unskilled laborer
- A172: Skilled laborer
- A174: Employee
- A171: Civil servant

In [13]:
credit_df['duration'].unique()

array([ 6, 48, 12, 42, 24, 36, 30, 15,  9, 10,  7, 60, 18, 45, 11, 27,  8,
       54, 20, 14, 33, 21, 16,  4, 47, 13, 22, 39, 28,  5, 26, 72, 40],
      dtype=int64)

In [15]:
credit_df['credit_history'].unique()

array(['A34', 'A32', 'A33', 'A30', 'A31'], dtype=object)

- A34: No credit history
- A32: Less than 0.5 years of credit history
- A33: 0.5 to 1.5 years of credit history
- A30: 1.5 to 4 years of credit history
- A31: More than 4 years of credit history

In [17]:
credit_df['savings_acc'].unique()

array(['A65', 'A61', 'A63', 'A64', 'A62'], dtype=object)

- A65: No savings account or negative savings
- A61: 0 <= Savings < 750 DM
- A63: 750 <= Savings < 1500 DM
- A64: 1500 <= Savings < 3000 DM
- A62: Savings >= 3000 DM

In [18]:
credit_df['present_emp_since'].unique()

array(['A75', 'A73', 'A74', 'A71', 'A72'], dtype=object)

- A75: Employed less than 1 year
- A73: Employed 1 to 4 years
- A74: Employed 5 to 7 years
- A71: Employed 8 to 10 years
-A72: Employed more than 10 years

In [19]:
credit_df['inst_rate'].unique()

array([4, 2, 3, 1], dtype=int64)

| Value	| Interpretation |
|-------|----------------|
| 1 | Low installment rate (< 20%) |
| 2	| Moderate installment rate (20% - 39%) |
| 3	| High installment rate (40% - 59%) |
| 4	| Very high installment rate (>= 60%) |

In [20]:
credit_df['personal_status'].unique()

array(['A93', 'A92', 'A91', 'A94'], dtype=object)

- A93: Married
- A92: Single (never married)
- A91: Divorced
- A94: Widowed

In [21]:
credit_df['residing_since'].unique()

array([4, 2, 3, 1], dtype=int64)

| Value | Description |
|-------|--------------|
| 1  | Less than 1 year |
| 2 | 1 to 4 years |
| 3 | 5 to 7 years |
| 4 | More than 7 years |

In [22]:
credit_df['age'].unique()

array([67, 22, 49, 45, 53, 35, 61, 28, 25, 24, 60, 32, 44, 31, 48, 26, 36,
       39, 42, 34, 63, 27, 30, 57, 33, 37, 58, 23, 29, 52, 50, 46, 51, 41,
       40, 66, 47, 56, 54, 20, 21, 38, 70, 65, 74, 68, 43, 55, 64, 75, 19,
       62, 59], dtype=int64)

In [23]:
credit_df['inst_plans'].unique()

array(['A143', 'A141', 'A142'], dtype=object)

| Value	| Interpretation |
|-------|----------------|
| A143	| No installment plans |
| A141	| Regular installment plans |
| A142	| Irregular installment plans |

In [24]:
credit_df['num_credits'].unique()

array([2, 1, 3, 4], dtype=int64)

| Value	| Interpretation |
|-------|----------------|
| 1	| 1 to 2 existing credits |
| 2	| 3 to 4 existing credits |
| 3	| 5 to 6 existing credits |
| 4	| More than 6 existing credits |

In [25]:
credit_df['status'].unique()

array([0, 1], dtype=int64)

| value | Interpretation |
|---|---|
| 0 | Good credit standing (paid off loan) |
| 1 | Bad credit standing (loan defaulted) |

In [26]:
credit_df.columns

Index(['checkin_acc', 'duration', 'credit_history', 'amount', 'savings_acc',
       'present_emp_since', 'inst_rate', 'personal_status', 'residing_since',
       'age', 'inst_plans', 'num_credits', 'job', 'status'],
      dtype='object')

In [27]:
credit_df.status.value_counts()

status
0    700
1    300
Name: count, dtype: int64

The output displays that there are 700 observations of good credit and 300 observations of bad credit.